<a href="https://colab.research.google.com/github/screid/Estadistica_Computacional_UGM/blob/main/Codigo_para_obtener_georeferencias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geopy

In [2]:
# Librerias
import pandas as pd
import numpy as np
import re
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [13]:
# Cargas el dataframe
# MODIFICAS AQUI Y  PONES EL NOMBRE DEL ARCHIVO
data = pd.read_csv('rubros_providencia.csv')

In [14]:
print("Número de filas: " , data.shape[0])

Número de filas:  65547


In [10]:
def escritura_y_transformacion_dataframe(data, comuna, region, n_partes):

  data['COMUNA'] = comuna
  data['REGION'] = region
  data['PAIS'] = 'Chile'

  lista_de_dataframes = np.array_split(data, n_partes)

  dataframe_dict = {}
  for i, df_parte in enumerate(lista_de_dataframes):
    dataframe_dict[f'df_parte_{i+1}'] = df_parte

  return dataframe_dict

In [6]:
def funcion_calculadora_de_lat_lon(data):

  # Función para limpiar la dirección
  def limpiar_direccion(direccion):
      # Convierte a mayúsculas por consistencia
      direccion = direccion.upper()
      # Elimina palabras como LOCAL, OFICINA, DEPTO, etc., seguidas de números u otros caracteres.
      # Si necesitas eliminar otras, colocalas siguiendo el patrón COSO1|COSO2. Este paso es importante, o la API no lo encuentra.
      direccion = re.sub(r'\b(LOCAL|OFICINA|DEPTO|DPTO|DEPARTAMENTO)\b[\s\S]*', '', direccion)
      # Elimina espacios finales
      return direccion.strip()

  # Construir dirección completa para geocodificar
  def construir_direccion_completa(row):
      base = limpiar_direccion(row['DIRECCION'])
      comuna = row['COMUNA']
      region = row['REGION']
      pais = row['PAIS']
      return f"{base}, {comuna}, {region}, {pais}"

  # Aplicar transformación
  data['DIRECCION_GEO'] = data.apply(construir_direccion_completa, axis=1)

  # Configurar geocoder
  geolocator = Nominatim(user_agent="macaco", timeout=5)
  geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, max_retries=1, error_wait_seconds=1.0)

  # Función de geocodificación robusta
  def obtener_coordenadas(direccion):
      try:
          ubicacion = geocode(direccion)
          if ubicacion:
              return pd.Series([ubicacion.latitude, ubicacion.longitude])
          else:
              return pd.Series([None, None])
      except Exception as e:
          print(f"Error con dirección '{direccion}': {e}")
          return pd.Series([None, None])

  # Aplicar
  data[['latitud', 'longitud']] = data['DIRECCION_GEO'].apply(obtener_coordenadas)

  return data

In [7]:
# MODIFICAS AQUI POR CADA UNO DE LAS DIVIONES QUE HICISTE. SI HICISTE 6, Y CUENTAS DESDE EL 0, SERIA DESDE DEL DATAFRAME_0 HASTA EL DATAFRAME_5
dataframe_1 = escritura_y_transformacion_dataframe(
    data,
    'Providencia',
    'Metropolitana',
    6)['df_parte_1']

resultado_dataframe1 = funcion_calculadora_de_lat_lon(dataframe_1)

resultado_dataframe1.to_csv('resultado_dataframe1.csv')

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,FECHA OTORGAMIENTO,ROL,NOMBRE,DIRECCION,GIRO O ACTIVIDAD,COMUNA,REGION,PAIS,DIRECCION_GEO,latitud,longitud
0,30/04/2025,2-201409,FUNERARIA IVÁN MARTÍNEZ IV SPA,SANTA MARIA 0180,OFICINA ADMINISTRATIVA DE SERVICIOS FUNERARIOS...,Providencia,Metropolitana,Chile,"SANTA MARIA 0180, Providencia, Metropolitana, ...",-33.420450,-70.614631
1,30/04/2025,2-201404,STUDIO GREECE SPA,AVENIDA NUEVA PROVIDENCIA 2606 LOCAL 4,"SERVICIOS DE BARBERÍA, PELUQUERÍA, MANICURA, V...",Providencia,Metropolitana,Chile,"AVENIDA NUEVA PROVIDENCIA 2606, Providencia, M...",-33.427879,-70.617317
2,30/04/2025,2-201405,APT COMERCIALIZADORA SPA,ANTONIO BELLET 193 OFICINA 1210,OFICINA TRIBUTARIA DE VENTA AL POR MENOR DE PR...,Providencia,Metropolitana,Chile,"ANTONIO BELLET 193, Providencia, Metropolitana...",-33.425374,-70.617923
3,30/04/2025,2-201406,VARQUEZ SPA,ANTONIO BELLET 193 OFICINA 1210,OFICINA TRIBUTARIA DE VENTA AL POR MENOR DE VE...,Providencia,Metropolitana,Chile,"ANTONIO BELLET 193, Providencia, Metropolitana...",-33.425374,-70.617923


In [9]:
# MODIFICAS AQUI POR CADA UNO DE LAS DIVIONES QUE HICISTE. SI HICISTE 6, Y CUENTAS DESDE EL 0, SERIA DESDE DEL DATAFRAME_0 HASTA EL DATAFRAME_5
dataframe_2 = escritura_y_transformacion_dataframe(
    data,
    'Providencia',
    'Metropolitana',
    6)['df_parte_2']

resultado_dataframe2 = funcion_calculadora_de_lat_lon(dataframe_2)

resultado_dataframe2.to_csv('resultado_dataframe2.csv')